<a href="https://colab.research.google.com/github/fernandopersan/4EscolaBigData/blob/main/FeatureExtraction1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extração de características com ResNet50 pré-treinada e Cross-Validation com K-NN**

Antes de iniciar a execução do código faça a seguinte alteração:
Menu: **Ambiente de execução -> Alterar o tipo de ambiente de execução -> GPU**

In [ ]:
import numpy as np
from PIL import Image

from keras.datasets import cifar10
from keras.models import Model
from keras.applications import resnet50

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

A função **lowSampleDataset(X,Y)** apenas reduz a quantidade de exemplos para que a execução seja mais rápida em nosso exemplo!

In [ ]:
def lowSampleDataset(X, Y):
    perm = np.random.permutation(X.shape[0])
    X = X[perm[0 : (int)(X.shape[0] * (5/100))]]
    Y = Y[perm[0 : (int)(Y.shape[0] * (5/100))]]
    return X, Y

**Pré-processamento:**

1.   Carregamos o dataset CIFAR10
2.   Reduzimos a quantidade de exemplos
1.   Alteramos a resolução dos exemplos

Vamos supor que, neste exemplo, não exista o conjunto de teste, ou seja, existe somente o conjunto de treinamento em nosso dataset!






In [ ]:
print("Loading CIFAR10 images ...")
(Xtrain, Ytrain), (Xtest, Ytest) = cifar10.load_data()

print('\tOriginal training set shape: ', Xtrain.shape)

Xtrain, Ytrain = lowSampleDataset(Xtrain, Ytrain)

X = []
for i in range(0, Xtrain.shape[0]):
    X.append(np.array(Image.fromarray(Xtrain[i]).resize(size=(224,224))))
Xtrain = np.array(X)

print('\tTraining set shape: ', Xtrain.shape)

Loading CIFAR10 images ...
	Original training set shape:  (50000, 32, 32, 3)
	Training set shape:  (2500, 224, 224, 3)


**Extração de características:**

Carregamos a ResNet50 e extraímos características utilizando a camada de pré-predição (-2)

In [ ]:
print("Loading the ResNet50-ImageNet model ...")
model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
#model.summary()

prediction = np.array(model.predict(Xtrain))
Xtrain = np.reshape(prediction, (prediction.shape[0], prediction.shape[1]))

print('\tFeatures training shape: ', Xtrain.shape)

Loading the ResNet50-ImageNet model ...
	Features training shape:  (2500, 2048)


**Classificação utilizando Cross-Validation e K-NN:**

Supondo que não exista conjuntos de treinamento e testes separados, a solução é aplicar Cross-Validation:


1.   StratifiedkFold: tenta balancear os conjuntos
2.   KFold: não possui a preocupação com o balanceamento dos conjuntos



In [ ]:
print("Cross-validation with K-NN ...")
kf = StratifiedKFold(n_splits=5, shuffle=True)
#kf = KFold(n_splits=5, shuffle=True)

knn = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(knn, Xtrain, np.ravel(Ytrain, order='C'), cv=kf)
print('\tAccuracy K-NN: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))

Cross-validation with K-NN ...
	Accuracy K-NN: 0.6580 +/- 0.0057
